In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
r = requests.get("https://spotifycharts.com/regional/us/daily/latest")

In [ ]:
r.headers["Content-Type"]

In [ ]:
html = BeautifulSoup(r.text, 'html.parser')

In [ ]:
title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
ids = html.find_all('td', attrs = {'class':'chart-table-image'})
ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]

In [ ]:
t = [{'title':i.strong.text,
      'artist':i.span.text.replace("by ", ""),
      'streams':int(j.text.replace(',', '')),
      'id':ids} for i, j, ids in zip(title_artist, streams, ids)]

# "Stream" will look like this

In [ ]:
from time import sleep
for song in t:
    print(song)
    sleep(1)

# Scraping Function

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep

def get_chart(region = 'global', date = 'latest'):
    
    # create url to query
    url = "https://spotifycharts.com/regional/{}/daily/{}".format(region, date)
    
    # run query, if fails, re-try after 3 seconds
    r = requests.get(url)
    while r.status_code != 200:
        print("Error parsing {}\nError code: {}\nTrying again...\n".format(url, r.status_code))
        sleep(5)
        r = requests.get(url)
    
    # Parse HTML response for data
    html = BeautifulSoup(r.text, 'html.parser')
    title_artist = html.find_all('td', attrs = {'class':'chart-table-track'})
    streams = html.find_all('td', attrs = {'class':'chart-table-streams'})
    ids = html.find_all('td', attrs = {'class':'chart-table-image'})
    ids = [i.a.get("href").replace("https://open.spotify.com/track/", "") for i in ids]
    
    # Create a list of dictionaries with the data
    dta =[{'title':i.strong.text,
          'artist':i.span.text.replace("by ", ""),
          'streams':int(j.text.replace(',', '')),
          'id':ids,
          'date':date,
          'region':region} for i, j, ids in zip(title_artist, streams, ids)]
    
    return dta

In [ ]:
data = get_chart(region = 'fr', date = "2019-03-16")
for i in data[0:10]:
    print(i)
    sleep(1)

In [ ]:
with open("FR-2017-10-19.txt", "a+") as f:
    for song in data:
        f.write(str(song) + "\n")

#### The below cell doesn't really create valid JSON - it's 200 JSONs appended in a file...

In [ ]:
import json
with open("FR-2017-10-19.json", "a+") as f:
    for song in data:
        json.dump(song, f)

# Publish to a Kafka topic

In [ ]:
import kafka
import json

In [ ]:
data[0]

In [ ]:
producer = kafka.KafkaProducer(value_serializer=lambda m: json.dumps(m).encode('ascii'), bootstrap_servers=['localhost:9092'])
for record in data:
    producer.send('TutorialTopic', record)
    sleep(1)

In [ ]:
# need to pass a JSON desserializer
consumer = kafka.KafkaConsumer('TutorialTopic', bootstrap_servers=['localhost:9092'], auto_offset_reset='earliest', enable_auto_commit=False)

In [ ]:
for message in consumer:
    print("%s:%d:%d: key=%s value=%s" % (message.topic, message.partition,
                                         message.offset, message.key,
                                         message.value))